In [ ]:
# Import necessary libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
from selenium_stealth import stealth 
import re
from datetime import datetime
from dateutil.relativedelta import relativedelta
import random

# Initialize Chrome options
chrome_options = Options()
time.sleep(random.uniform(6, 11)) 
# Set up date formatting for today's date
today = datetime.today().strftime('%Y-%m-%d')
#LinkedIn Credentials
username=""
password=""
time.sleep(random.uniform(8, 12)) 

    
#Url of the Linkedin Page you want to scrape
page = 'https://www.linkedin.com/feed/'#access Webriver
browser = webdriver.Chrome()
# Appliquer les paramètres stealth pour masquer les comportements de bot
stealth(browser,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True)
time.sleep(random.uniform(6, 11)) 
#Open login page
browser.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')
time.sleep(random.uniform(4, 10)) 
#Go to webpage
post_page = page 
#+ '/posts'
#post_page = post_page.replace('//posts','/posts')
browser.get(post_page)


: 

In [2]:
if page[-1] == "/":
    company_name = page.split("/")[-2]
else:
    company_name = page.split("/")[-1]
    
company_name = company_name.replace('-',' ').title()
print(company_name)

Feed


In [9]:
SCROLL_PAUSE_TIME = random.uniform(1, 8)
MAX_SCROLLS = None  # Utilise None pour indiquer qu'il n'y a pas de limite

# JavaScript commands
SCROLL_COMMAND = "window.scrollTo(0, document.body.scrollHeight);"
GET_SCROLL_HEIGHT_COMMAND = "return document.body.scrollHeight"

# Attendre quelques secondes pour le chargement de la page après connexion
time.sleep(random.uniform(8, 12))

# Initial scroll height
last_height = browser.execute_script(GET_SCROLL_HEIGHT_COMMAND)
scrolls = 0
no_change_count = 0

while True:
    # Scroll down to bottom
    browser.execute_script(SCROLL_COMMAND)

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script(GET_SCROLL_HEIGHT_COMMAND)

    # Increment no change count or reset it
    no_change_count = no_change_count + 1 if new_height == last_height else 0

    # Break loop if the scroll height has not changed for 3 cycles or reached the maximum scrolls
    if no_change_count >= 3 or (MAX_SCROLLS is not None and scrolls >= MAX_SCROLLS):
        break

    # Attendre quelques secondes pour le chargement de la page après connexion
    time.sleep(random.uniform(8, 12))

    last_height = new_height
    scrolls += 1
 


In [10]:
# Récupération du code source de la page LinkedIn
company_page = browser.page_source

# Parsing du HTML avec BeautifulSoup
linkedin_soup = bs(company_page, "html.parser")  # Pas besoin d'encoder en UTF-8 ici

# Sauvegarde du contenu HTML de la page dans un fichier
with open(f"{company_name}_soup.txt", "w+", encoding="utf-8") as t:
    t.write(linkedin_soup.prettify())

# Recherche des publications (div avec la classe "feed-shared-update-v2")
containers = linkedin_soup.find_all("div", {"class": "feed-shared-update-v2"})

# Filtrage des containers qui contiennent un attribut "data-urn" avec "activity"
containers = [container for container in containers if 'activity' in container.get('data-urn', '')]

# Affichage du nombre de publications trouvées
print(len(containers))

# Sauvegarde des publications filtrées dans un fichier
containers_text = "\n\n".join([c.prettify() for c in containers])
with open(f"{company_name}_soup_containers.txt", "w+", encoding="utf-8") as t:
    t.write(containers_text)



8


In [11]:
# Charger le HTML à partir du fichier (si nécessaire)
with open(f"{company_name}_soup_containers.txt", "r", encoding="utf-8") as f:
    html_content = f.read()

# Parser le HTML
linkedin_soup = bs(html_content, "html.parser")

# Trouver tous les posts
containers = linkedin_soup.find_all("div", {"class": "feed-shared-update-v2"})

# Extraire le HTML de chaque post et l'encapsuler proprement
posts_data = [{"Post HTML": f'"""{str(container)}"""'} for container in containers]

# Créer un DataFrame Pandas
df_posts = pd.DataFrame(posts_data)

# Sauvegarder en CSV avec un bon séparateur et gestion des guillemets
df_posts.to_csv(f"{company_name}_posts.csv", index=False, encoding="utf-8", sep=";", quoting=1)

print(f"✅ Extraction terminée ! {len(df_posts)} posts enregistrés dans {company_name}_posts.csv")


✅ Extraction terminée ! 8 posts enregistrés dans Feed_posts.csv


In [12]:
def get_actual_date(date):
    today = datetime.today().strftime('%Y-%m-%d')
    current_year = datetime.today().strftime('%Y')
    
    def get_past_date(days=0, weeks=0, months=0, years=0):
        date_format = '%Y-%m-%d'
        dtObj = datetime.strptime(today, date_format)
        past_date = dtObj - relativedelta(days=days, weeks=weeks, months=months, years=years)
        past_date_str = past_date.strftime(date_format)
        return past_date_str

    past_date = date
    
    if 'hour' in date:
        past_date = today
    elif 'day' in date:
        date.split(" ")[0]
        past_date = get_past_date(days=int(date.split(" ")[0]))
    elif 'week' in date:
        past_date = get_past_date(weeks=int(date.split(" ")[0]))
    elif 'month' in date:
        past_date = get_past_date(months=int(date.split(" ")[0]))
    elif 'year' in date:
        past_date = get_past_date(months=int(date.split(" ")[0]))
    else:
        split_date = date.split("-")
        if len(split_date) == 2:
            past_month = split_date[0]
            past_day =  split_date[1]
            if len(past_month) < 2:
                past_month = "0"+past_month
            if len(past_day) < 2:
                past_day = "0"+past_day
            past_date = f"{current_year}-{past_month}-{past_day}"
        elif len(split_date) == 3:
            past_month = split_date[0]
            past_day =  split_date[1]
            past_year = split_date[2]
            if len(past_month) < 2:
                past_month = "0"+past_month
            if len(past_day) < 2:
                past_day = "0"+past_day
            past_date = f"{past_year}-{past_month}-{past_day}"

    return past_date



def convert_abbreviated_to_number(s):
    if 'K' in s:
        return int(float(s.replace('K', '')) * 1000)
    elif 'M' in s:
        return int(float(s.replace('M', '')) * 1000000)
    else:
        return int(s)


In [13]:
def clean_date(date_str):
    """Nettoie la chaîne de date pour ne garder que l'heure ou la date."""
    if not date_str or date_str == "No Content":
        return "Unknown"
    cleaned_date = re.sub(r"•\s*Modifié\s*•", "", date_str).strip()
    return cleaned_date

In [14]:
import re
from bs4 import BeautifulSoup

# Fonction pour vérifier si un post est une offre de travail
def is_job_post(post_text):
    """Vérifie si le texte du post contient des mots-clés liés aux offres d'emploi."""
    job_keywords = [
    "recrutement", "offre d'emploi", "hiring", "job", "emploi", "poste", "recherche", "candidature",
    "recrute", "opportunité", "carrière", "join us", "we are hiring", "apply now",
    "travail", "embauche", "job opportunity", "job opening", "job vacancy", "job search",
    "job offer", "job posting", "job application", "job board", "job portal", "job fair",
    "job market", "job seeker", "job hunter", "job alert", "job description", "job title",
    "job interview", "job contract", "job placement", "job agency", "job recruitment",
    "job skills", "job requirements", "job qualifications", "job benefits", "job salary",
    "job location", "job type", "full-time", "part-time", "remote job", "freelance", "internship",
    "stage", "temporary job", "permanent job", "contract job", "work from home", "telecommuting",
    "career opportunity", "career development", "career growth", "career change", "career path",
    "career advice", "career coach", "career counseling", "career transition", "career planning",
    "employment agency", "employment opportunity", "employment contract", "employment law",
    "employment benefits", "employment rate", "employment status", "employment history",
    "employment gap", "employment test", "employment verification", "employment screening",
    "talent acquisition", "talent pool", "talent management", "talent search", "talent pipeline",
    "talent development", "talent retention", "talent strategy", "talent assessment", "talent network",
    "recruitment agency", "recruitment process", "recruitment strategy", "recruitment campaign",
    "recruitment consultant", "recruitment software", "recruitment tools", "recruitment metrics",
    "recruitment trends", "recruitment challenges", "recruitment solutions", "recruitment advertising",
    "recruitment events", "recruitment drive", "recruitment fair", "recruitment webinar",
    "recruitment webinar", "recruitment video", "recruitment branding", "recruitment marketing",
    "recruitment analytics", "recruitment automation", "recruitment outsourcing", "recruitment technology",
    "recruitment platform", "recruitment system", "recruitment database", "recruitment CRM",
    "recruitment AI", "recruitment chatbot", "recruitment AI tools", "recruitment AI solutions",
    "recruitment AI platform", "recruitment AI software", "recruitment AI technology",
    "recruitment AI system", "recruitment AI database", "recruitment AI CRM", "recruitment AI chatbot"
]
    # Convertit le texte en minuscules pour une recherche insensible à la casse
    post_text_lower = post_text.lower()
    # Vérifie si l'un des mots-clés est présent dans le texte
    return any(keyword in post_text_lower for keyword in job_keywords)

# Fonction pour extraire le texte d'un élément HTML donné
def get_text(container, selectors):
    """Tente d'extraire du texte à partir d'une liste de sélecteurs."""
    for selector, attributes in selectors:
        element = container.find(selector, attributes)
        if element:
            text = element.get_text(strip=True)
            if text:
                return text
    return "No Content"

# Fonction pour extraire le nom de la société ou de la personne
def get_author_name(container):
    """Extrait le nom de la société ou de la personne qui a publié le post."""
    author_selectors = [
        ("span", {"class": "update-components-actor__title"}),
        ("span", {"class": "update-components-actor__name"}),
        ("a", {"class": "update-components-actor__name-link"})
    ]
    return get_text(container, author_selectors)

# Fonction pour extraire la date du post
def get_post_date(container):
    """Extrait la date de publication du post."""
    date_selectors = [
        ("time", {}),  
        ("span", {"class": "update-components-actor__sub-description"})
    ]
    return get_text(container, date_selectors)

# Boucle principale pour traiter chaque publication trouvée dans la page
for container in containers:
    # Extraction du texte du post
    post_text = get_text(container, [
        ("div", {"class": "feed-shared-update-v2__description-wrapper"}),
        ("div", {"class": "update-components-text"}),
        ("span", {"class": "break-words"})
    ])

    # Vérifie si le post est une offre de travail
    if not is_job_post(post_text):
        continue  # Passe au post suivant si ce n'est pas une offre de travail

    # Extraction du nom de la société ou de la personne
    author_name = get_author_name(container)

    # Extraction de la date du post
    post_date = get_post_date(container)

    # Création du dictionnaire contenant les données du post
    post_data = {
        "Auteur": author_name,
        "Date": post_date,
        "Post Text": post_text
    }

    # Affichage des données extraites
    print(f"\n  Offres  : {post_data}")


  Offres  : {'Auteur': 'Farah Ben LaghaFarah Ben Lagha• Abonné• Abonné', 'Date': '1 j •Il y a 1 jour', 'Post Text': '𝗡𝗼𝘂𝘀 𝗿𝗲𝗰𝗿𝘂𝘁𝗼𝗻𝘀 𝗱𝗲𝘀 𝗷𝗲𝘂𝗻𝗲𝘀 𝘁𝗮𝗹𝗲𝗻𝘁𝘀 𝗳𝗿𝗮𝗶𝗰𝗵𝗲𝗺𝗲𝗻𝘁 𝗱𝗶𝗽𝗹𝗼̂𝗺𝗲́𝘀 ✈👉𝐃𝐞𝐬𝐬𝐢𝐧𝐚𝐭𝐞𝐮𝐫(𝐭𝐫𝐢𝐜𝐞)◾https://lnkd.in/dYMZGTNj👉𝐏𝐫𝐨𝐣𝐞𝐭𝐞𝐮𝐫 É𝐥𝐞𝐜𝐭𝐫𝐢𝐪𝐮𝐞◾https://lnkd.in/dRXNBWKU👉 𝐏𝐫𝐨𝐣𝐞𝐭𝐞𝐮𝐫◾https://lnkd.in/dkwWsrdf◾https://lnkd.in/d4izWUvSDécolle ta carrière avecSafran! Postule maintenant et fais partie de notre aventure ✈'}

  Offres  : {'Auteur': 'Guillaume GuerraGuillaume Guerra• 3e et +• 3e et +', 'Date': '1 j • Modifié •Il y a 1 jour', 'Post Text': 'Pas besoin d’un Bac+5 pour avancer dans sa carrière.7 qualités essentielles pour faire la différence.Ces 7 qualités, attitudes, savoir être souvent négligées, peuvent faire toute la différence dans votre carrière et peuvent vous aider à vous démarquer.Elles ne demandent aucun diplôme, aucune expertise technique juste de la volonté et de la discipline.Elles sont accessibles à toutes et tous.💡 7 compétences comportementales qui ne demandent aucun tale